In [1]:
using MaximumEntropy

[ Info: Precompiling MaximumEntropy [3009f659-cc7f-4c8f-93f4-1926c60516fb] (cache misses: include_dependency fsize change (2), dep missing source (2), include_dependency fhash change (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing MaximumEntropy [3009f659-cc7f-4c8f-93f4-1926c60516fb].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.


In [ ]:

gpp = -u[2]^2*(g1+g2)*(mt*cosh(etap-eta))^2-2*u[1]*u[2]*(g1+g2)*mt*pt*cosh(etap-eta)*cos(phip-phi)-u[1]^2*(g1+g2)*(pt*cos(phip-phi))^2
ur = 0.1
etap=0.0
phip=0.0
m = 0.5
mult_E=1.0
eta = 0.0
phi = 0.0
        
mult_P=1.0
pt=2.0
mult_shear=1.0
g1= 0.1
g2 =0.1


energy_density(ur,m,mult_E,mult_P)[1]
shear(ur,m,mult_E,mult_P,g1,g2)[1][1]
shear(ur,m,mult_E,mult_P,g1,g2)[2][1]
shear(ur,m,mult_E,mult_P,g1,g2)


@show res(2)[:]
sol1,sol2=distr_function(resultNofo1,discretization,1.0,fluidproperty)
sol1

In [ ]:

#define the grid
rmax = 30;
gridpoints=400;
grid_params = Fluidum.GridParameters(rmax, gridpoints);

#define the detector to use 
det = :ALICE;
det_name = detector_dict[det].name; 
dσ_QQdy = detector_dict[det].dσ_QQdy;
σ_in = detector_dict[det].σ_in;

#centrality classes
cent1 = 0;
cent2 = 10;

#define the initial parameters
tspan = (0.4,12);
Norm = 90.; 
tau0 = tspan[1];
tau_fs = 0.1;
rdrop = 8.5;
initial_params = Fluidum.InitialParameters(Norm, tau0, tau_fs, rdrop);

In [ ]:

#take the tabulated data 
ent = TabulatedData(string(@__DIR__, "\\EntropyProfile_", det_name, ".txt"));
ncoll = TabulatedData(string(@__DIR__, "\\NcollProfile_", det_name, ".txt"));
fonll = TabulatedData(string(@__DIR__, "\\FONLL_", det_name, ".txt"));


In [ ]:
fields, ccbar = initialize_fields(ent, ncoll, fonll, cent1, cent2; grid_params, initial_params, dσ_QQdy, m=1.5);
eos = Heavy_Quark(readresonancelist(), ccbar)


In [ ]:
viscosity = QGPViscosity(0.1,0.2); #or, ZeroViscosity();
bulk = SimpleBulkViscosity(0.083,15.0); #or, ZeroBulkViscosity();

diffusion_Ds = HQdiffusion(0.1, 1.5);
params_Ds=Fluidum.FluidProperties(eos,viscosity,bulk,diffusion_Ds);

In [ ]:
fo_Ds = freeze_out_routine(fields.discrete_field, Fluidum.matrxi1d_visc_HQ!, params_Ds, fields.initial_field, tspan; Tfo = 0.156);
field_results_Ds = oneshoot(fields.discrete_field, Fluidum.matrxi1d_visc_HQ!, params, fields.initial_field, tspan; reltol = 1e-8);


In [ ]:

plot_fields_evolution(field_results_Ds, fields.discretization, tspan, params_Ds)
